# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
!pip install textdistance


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import textdistance
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from difflib import get_close_matches

In [2]:
# библиотека для отслеживания прогресса
from tqdm.notebook import tqdm

In [3]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


### Без ранжирования равнозначных по расстоянию редактирования кандидатов по вероятности (с семинара)

In [4]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [5]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [6]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [7]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.12886443221610805


In [8]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

In [9]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [10]:
vocab.most_common(10)

[('в', 267296),
 ('и', 147115),
 ('на', 81926),
 ('с', 61681),
 ('года', 43894),
 ('по', 37235),
 ('году', 32197),
 ('из', 29150),
 ('был', 23293),
 ('не', 23228)]

In [11]:
# поиск ближайшего слова по косинусному расстоянию
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [12]:
%%time
get_closest_match_vec('сонце', X, vec) # это расстояние - чем меньше тем лучше

CPU times: user 202 ms, sys: 99.4 ms, total: 302 ms
Wall time: 536 ms


[('цоссен', 0.05131670194948623),
 ('цоссене', 0.05612019255146106),
 ('концессионное', 0.08649972160886044),
 ('солнце', 0.0871290708247231),
 ('сценок', 0.0871290708247231),
 ('сценой', 0.0871290708247231),
 ('сосновец', 0.09630388588493599),
 ('сено', 0.10557280900008414),
 ('носе', 0.10557280900008414),
 ('броненосец', 0.10557280900008414),
 ('сенокосцев', 0.10557280900008414),
 ('соне', 0.10557280900008414),
 ('бессоннице', 0.10557280900008414),
 ('сное', 0.10557280900008414),
 ('бесценного', 0.10557280900008414),
 ('сцен', 0.10557280900008414),
 ('броненосце', 0.10557280900008414),
 ('онсе', 0.10557280900008414),
 ('лентоносец', 0.10557280900008414),
 ('осен', 0.10557280900008414)]

In [13]:
# поиск наиболее похожего слова по расстоянию Левенштейна
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [15]:
%%time
get_closest_match_with_metric('сонце', word2id, topn=20, metric=textdistance.levenshtein) # это расстояние - чем меньше тем лучше

CPU times: user 24.6 s, sys: 313 ms, total: 24.9 s
Wall time: 25.5 s


[('солнце', 0.8333333333333334),
 ('конце', 0.8),
 ('монце', 0.8),
 ('соне', 0.8),
 ('сонче', 0.8),
 ('донце', 0.8),
 ('солнцем', 0.7142857142857143),
 ('солнцев', 0.7142857142857143),
 ('солнца', 0.6666666666666667),
 ('сочное', 0.6666666666666667),
 ('синице', 0.6666666666666667),
 ('солнцу', 0.6666666666666667),
 ('сочные', 0.6666666666666667),
 ('свинце', 0.6666666666666667),
 ('сфорце', 0.6666666666666667),
 ('монцей', 0.6666666666666667),
 ('ньонце', 0.6666666666666667),
 ('соньер', 0.6666666666666667),
 ('сорное', 0.6666666666666667),
 ('донцем', 0.6666666666666667)]

In [16]:
# гибридный поиск наиболее похожего слова
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [17]:
%%time
get_closest_hybrid_match('сонце', X, vec, topn=5, metric=textdistance.damerau_levenshtein)

CPU times: user 177 ms, sys: 19.8 ms, total: 197 ms
Wall time: 222 ms


[('солнце', 0.8333333333333334),
 ('соне', 0.8),
 ('сное', 0.6),
 ('онсе', 0.6),
 ('сосновец', 0.5)]

In [18]:
# общее количество слов в словаре
N = sum(vocab.values())


# вероятность слова на основе его относительной частоты в словаре
def P(word, N=N):
    return vocab[word] / N

# возвращаем 1, если слово отсутствует в словаре vocab (то есть считается ошибкой) и 0, если слово присутствует в словаре.
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [19]:
# оценка качества функции get_closest_hybrid_match (без ранжирования равнозначных по расстоянию редактирования кандидатов по вероятности)
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

  0%|          | 0/915 [00:00<?, ?it/s]

0.847023511755878
0.421583850931677
0.09004249454461927


### Выбор наиболее вероятного слова из кандидатов с одинаковым расстоянием редактирования

In [30]:
def get_closest_hybrid_match_ranked(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein, vocab=vocab):
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    # Создаем список для хранения кандидатов с одинаковым расстоянием редактирования
    candidates_with_equal_distance = []

    # Ищем n первых кандидатов с одинаковым расстоянием редактирования
    for n in range(1, len(closest) + 1):
        equal_distance = len(set(dist for _, dist in closest[:n])) == 1
        if equal_distance:
            # Сохраняем их по очереди в список, пока не найдем отличие в расстоянии
            candidates_with_equal_distance.append(closest[n - 1])
        else:
            break

    if candidates_with_equal_distance:  # Если есть кандидаты с одинаковым расстоянием
        # Добавляем их вероятности
        candidates_with_equal_distance = [(word, dist, P(word)) for word, dist in candidates_with_equal_distance]
        # Ранжируем список по вероятности
        candidates_with_equal_distance.sort(key=lambda x: x[2], reverse=True)

        # Получаем оставшихся кандидатов
        remaining_candidates = closest[len(candidates_with_equal_distance):]

        # Объединяем первых ранжированных по вероятностям кандидатов с остальными
        merged_candidates = candidates_with_equal_distance + remaining_candidates

        return merged_candidates
    else:
        return closest  # Возвращаем исходный список, если не нашли кандидатов с одинаковым расстоянием


In [31]:
%%time
get_closest_hybrid_match('привт', X, vec, topn=5, metric=textdistance.damerau_levenshtein)


CPU times: user 205 ms, sys: 142 ms, total: 346 ms
Wall time: 750 ms


[('приват', 0.8333333333333334),
 ('привет', 0.8333333333333334),
 ('привить', 0.7142857142857143),
 ('привито', 0.7142857142857143),
 ('правит', 0.6666666666666667)]

In [32]:
# "приват" и "привет" – одинаковое расстояние редактирования, при этом "приват" – первый кандидат, тк встречается чаще в словаре
get_closest_hybrid_match_ranked('привт', X, vec, topn=5, metric=textdistance.damerau_levenshtein, vocab=vocab)

[('приват', 0.8333333333333334, 3.685543243254777e-06),
 ('привет', 0.8333333333333334, 2.7156634423982565e-06),
 ('привить', 0.7142857142857143),
 ('привито', 0.7142857142857143),
 ('правит', 0.6666666666666667)]

In [33]:
%%time
get_closest_hybrid_match('солнц', X, vec, topn=5, metric=textdistance.damerau_levenshtein)

CPU times: user 213 ms, sys: 82.1 ms, total: 295 ms
Wall time: 421 ms


[('солнце', 0.8333333333333334),
 ('солнцу', 0.8333333333333334),
 ('солнца', 0.8333333333333334),
 ('солонцы', 0.7142857142857143),
 ('солнцем', 0.7142857142857143)]

In [34]:
P("солнца")

3.220000938843647e-05

In [35]:
P("солнце")

2.444097098158431e-05

In [36]:
P("солнца") > P("солнце")

True

In [37]:
# "солнца" вероятнее, чем "солнце"
get_closest_hybrid_match_ranked('солнц', X, vec, topn=4, metric=textdistance.damerau_levenshtein, vocab=vocab)

[('солнца', 0.8333333333333334, 3.220000938843647e-05),
 ('солнце', 0.8333333333333334, 2.444097098158431e-05),
 ('солнцу', 0.8333333333333334, 3.103615362740865e-06),
 ('солонцы', 0.7142857142857143)]

In [38]:
# оценка качества функции get_closest_hybrid_match_ranked (с ранжированием по вероятности)
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match_ranked(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

  0%|          | 0/915 [00:00<?, ?it/s]

0.848224112056028
0.4309006211180124
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

#### 1)  Составляется словарь правильных слов

In [39]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [40]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [41]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall(r'\w+', corpus.lower()))

print(len(vocab)) 

368801


#### 2)  Создание словаря удалений: Для каждого слова из словаря правильных слов создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 

In [42]:
def edits1(word):
    letters = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return deletes

# Создаем словарь удалений на основе словаря правильных слов
delete_dict = {}
for word in vocab:
    deletions = edits1(word)
    for deletion in deletions:
        if deletion not in delete_dict:
            delete_dict[deletion] = word

print(len(delete_dict))

2872451


In [61]:
edits1("ноыостройка")

['оыостройка',
 'ныостройка',
 'ноостройка',
 'ноыстройка',
 'ноыотройка',
 'ноыосройка',
 'ноыостойка',
 'ноыострйка',
 'ноыострока',
 'ноыостройа',
 'ноыостройк']

In [43]:
from itertools import islice

for key in islice(delete_dict.items(), 20):
    print(key)

('овостройка', 'новостройка')
('нвостройка', 'новостройка')
('ноостройка', 'новостройка')
('новстройка', 'новостройка')
('новотройка', 'новостройка')
('новосройка', 'новостройка')
('новостойка', 'новостройка')
('новострйка', 'новостройка')
('новострока', 'новостройка')
('новостройа', 'новостройка')
('новостройк', 'новостройка')
('ижегородская', 'нижегородская')
('нжегородская', 'нижегородская')
('ниегородская', 'нижегородская')
('нижгородская', 'нижегородская')
('нижеородская', 'нижегородская')
('нижегродская', 'нижегородская')
('нижегоодская', 'нижегородская')
('нижегордская', 'нижегородская')
('нижегороская', 'нижегородская')


#### 3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
#### 4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово 

In [54]:
def correct_spelling_with_probability(word, delete_dict, vocab):
    # Генерируем варианты удалений для слова с опечаткой
    word_deletes = edits1(word)
    
    # Создаем пустой список для потенциальных исправлений
    possible_corrections = []
    
    # Итерируемся по вариантам удалений слова с опечаткой
    for delete in word_deletes:
        # проверяем, есть ли данный вариант удаления в словаре удалений
        if delete in delete_dict:
            # Если есть, добавляем соответствующее правильное слово в список потенциальных исправлений
            possible_corrections.append(delete_dict[delete])
    
    # Если есть потенциальные исправления
    if possible_corrections:
        # Выбираем наиболее вероятное правильное слово на основе его вероятности в словаре
        corrected_word = max(possible_corrections, key=lambda x: P(x, N=N))
        return corrected_word  # Возвращаем наиболее вероятное исправление
    
    # Если нет потенциальных исправлений, возвращаем исходное слово
    return word


In [55]:
%%time
correct_spelling_with_probability('иовостройка', delete_dict, vocab)

CPU times: user 64 µs, sys: 281 µs, total: 345 µs
Wall time: 360 µs


'новостройка'

In [56]:
%%time
correct_spelling_with_probability('солнке', delete_dict, vocab)

CPU times: user 77 µs, sys: 242 µs, total: 319 µs
Wall time: 331 µs


'солнце'

In [57]:
%%time
correct_spelling_with_probability('козырт', delete_dict, vocab)

CPU times: user 34 µs, sys: 111 µs, total: 145 µs
Wall time: 150 µs


'козырь'

In [58]:
%%time
correct_spelling_with_probability('сонце', delete_dict, vocab)

CPU times: user 50 µs, sys: 204 µs, total: 254 µs
Wall time: 259 µs


'конце'

In [59]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correct_spelling_with_probability(pair[1], delete_dict, vocab))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1


  0%|          | 0/915 [00:00<?, ?it/s]

In [60]:
print(correct/total) # процент правильных 
print(mistaken_fixed/total_mistaken) # сколько мы правильно ошибок исправили (только половину)
print(correct_broken/total_correct) # сколько раз исправляем правильные слова на неправильные

0.511255627813907
0.20263975155279504
0.4430917652463535
